In [7]:
inputFolder=""
folderCounter=1

In [1]:
# debug cell
with open('simple_table.txt', 'r') as f:
    print(type(f))
    cnt = 2
    for i,line in enumerate(f):
        print(i, ':\t', line)
        cnt -= 1
        if cnt == 0:
            break

<class '_io.TextIOWrapper'>
0 :	 CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO:AF	UKB-b-6134:ES	UKB-b-6134:SE	UKB-b-6134:LP	UKB-b-6134:AF	UKB-b-6134:SS	UKB-b-6134:EZ	UKB-b-6134:SI	UKB-b-6134:NC	UKB-b-6134:ID

1 :	 1	49298	rs10399793	T	C	.	PASS	0.623238	0.0017795	0.00366743	0.200659	0.623238	.	.	.	.	rs10399793



In [1]:
# debug cell
import gzip
with gzip.open('./ukb-b-6134.vcf.gz') as f:
    print(f)

FileNotFoundError: [Errno 2] No such file or directory: './ukb-b-6134.vcf.gz'

In [1]:
# to rerun any part of this code
import os
import json
import pickle
import time
import gzip
import shutil

In [12]:


def generate_gwas_output_as_tsv_file(inVCF_File, writeFile=None):
    root = './tabular_format_gwas_data/'
    t0 = time.time()
    if writeFile is None:
        writeFile = root + inVCF_File.split('/')[-1].split('.')[0] + '.tsv'
    
    print("Processing:", inVCF_File)
    if ".gz" == inVCF_File[-3:]:
        with gzip.open(inVCF_File, 'rb') as f_in:
            with open(root+'temp_vcf.vcf', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        inVCF_File = root+'temp_vcf.vcf'
    print("gzip open done :", inVCF_File)
    """
    Convert all the VCF into simplified tsv format
    Source: https://github.com/everestial/VCF-simplify#table-of-contents
    python3 ./VCF-Simplify/VCF-Simplify/VcfSimplify.py SimplifyVCF -toType table -inVCF ./ukb-b-6134.vcf -outFile ./simple_table.txt
    """
    vcf_simplify_path = "./VCF-Simplify/VCF-Simplify/VcfSimplify.py"
    out_File = root+'temp_table.tsv'
    os.system(f"python3 {vcf_simplify_path} SimplifyVCF -toType table -inVCF {inVCF_File} -outFile {out_File}")
    print(writeFile)
        
    f_read = open(out_File, 'r')
    f_write = open(writeFile, 'w')
    cnt = 0
    for line in f_read:
#         print(line)
        newliner = ''
        if '\n' == line[-1]:
            newliner = '\n'
            line = line.strip()
        if 'CHROM' in line:
            line = line.upper() + '\tPVAL_generated_from_LP'
        else:
            LP = float(line.split('\t')[10])
#             LP = float(line.split('\t')[14])
            #find out LP column from the tsv file
            
            p_val = str(10**(-1 * LP))
            line = line + '\t' + p_val
        line = line + newliner
    #     print(line)
        f_write.write(line)
        cnt += 1
    #     if cnt == 20: break

    f_read.close()
    f_write.close()
    print(f'Total SNPs: {cnt} | Total exec time: {time.time() - t0} seconds')

In [10]:
import pandas as pd
df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
df
ALL_GWAS_IDS = list(df['GWAS_ID'].to_numpy().tolist()) 


In [13]:
for GWAS_ID in ALL_GWAS_IDS:

    generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/'+GWAS_ID+'.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-12417.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-11495.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-12493.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-19732.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-17006.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-5779.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-15541.vcf.gz', writeFile=None)
# need to just specify the file path here
# ebi-a-GCST005920.vcf.gz
# ebi-a-GCST005923.vcf.gz
# ukb-b-14699.vcf.gz
# ukb-b-323.vcf.gz
# ukb-b-6358
# next update in the 'GWAS_datasets_to_consider.xlsx'


Processing: ./tabular_format_gwas_data/ukb-b-13806.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO:AF', 'UKB-b-13806:ES', 'UKB-b-13806:SE', 'UKB-b-13806:LP', 'UK

Total SNPs: 12013477 | Total exec time: 120.19564819335938 seconds
Processing: ./tabular_format_gwas_data/met-d-Total_C.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER',

Total SNPs: 12321876 | Total exec time: 116.55932235717773 seconds
Processing: ./tabular_format_gwas_data/ieu-b-25.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INF

Total SNPs: 10545187 | Total exec time: 96.35626554489136 seconds
Processing: ./tabular_format_gwas_data/ukb-b-12064.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'I

Total SNPs: 9851867 | Total exec time: 94.31156206130981 seconds
Processing: ./tabular_format_gwas_data/ukb-b-2209.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INF

Total SNPs: 9756890 | Total exec time: 95.25070977210999 seconds
Processing: ./tabular_format_gwas_data/ukb-b-6324.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INF

Total SNPs: 8862762 | Total exec time: 88.1577959060669 seconds
Processing: ./tabular_format_gwas_data/ukb-b-323.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO:

Total SNPs: 5294160 | Total exec time: 49.026020526885986 seconds
Processing: ./tabular_format_gwas_data/ukb-b-6358.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'IN

Total SNPs: 4801286 | Total exec time: 47.28146028518677 seconds
Processing: ./tabular_format_gwas_data/ukb-b-19732.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'IN

Total SNPs: 9851867 | Total exec time: 94.66067051887512 seconds
Processing: ./tabular_format_gwas_data/ukb-b-12417.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'IN

In [ ]:

# ' abc def\n'.strip()

In [ ]:
# 10**(-1*0.200659) #0.6300007

In [17]:
# %pip install openpyxl
import pandas as pd

category_map = {
    'Continuous' : 'beta', 
    'Categorical Ordered (assumed continuous)': 'beta',
    'Binary': 'or',
    'NA (Possibly binary)': 'or'
}


df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
# print(df)
# print(df['Category'].unique())
# GWAS_ID = 'ieu-b-111'
# category = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
# category

In [18]:
from glob import glob
# df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
# all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
# ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257','ukb-b-2209','ukb-b-17627','ieub-109','ieu-b-110','ieu-b-111','met-d-Total_C'})
# bbj-a-78,ukb-a-257,bbj-a-46
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78'})
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {  'ieu-b-109','ieu-b-110','ieu-b-111','met-d-Total_C','ukb-b-2209','ukb-b-17627','ukb-a-257','ieu-a-1283','bbj-a-46', 'bbj-a-78','ukb-b-14180','ukb-b-13806','ukb-b-12064','ukb-b-323','ukb-b-14699','ukb-b-14699','ukb-b-5779'})
ALL_GWAS_IDS = list(df['GWAS_ID'].to_numpy().tolist()) 

print(len(ALL_GWAS_IDS))
print(ALL_GWAS_IDS)

base_files = {}

for GWAS_ID in ALL_GWAS_IDS:
    
#     file = ROOT + GWAS_ID + '.vcf.gz'
#     base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]]
    base_files[GWAS_ID] = df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]
#     base_files[GWAS_ID]=category_map[base_files[GWAS_ID]]
    
print(base_files)
#     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)
# print(name_mapping)
# print(name_mapping)


39
['ukb-b-13806', 'ukb-d-20405_0', 'ukb-d-20405_1', 'ukb-d-20405_2', 'met-d-Total_C', 'ieu-b-109', 'ieu-b-110', 'ieu-b-111', 'ieu-b-25', 'ieu-b-38', 'ieu-b-39', 'ukb-a-257', 'ukb-b-12064', 'ukb-b-17627', 'ukb-b-18275', 'ukb-b-19953', 'ukb-b-2209', 'ukb-b-3957', 'ukb-b-4424', 'ukb-b-6134', 'ukb-b-6324', 'ukb-b-7663', 'ukb-b-770', 'ukb-b-8476', 'ukb-b-323', 'ukb-b-14699', 'ukb-b-14180', 'ukb-b-17243', 'ukb-b-6358', 'ukb-b-17006', 'ukb-b-5779', 'ukb-b-15541', 'ukb-b-19732', 'ukb-b-20289', 'ukb-b-14057', 'ukb-b-12963', 'ukb-b-12417', 'ukb-b-11495', 'ukb-b-12493']
{'ukb-b-13806': 'or', 'ukb-d-20405_0': 'or', 'ukb-d-20405_1': 'or', 'ukb-d-20405_2': 'or', 'met-d-Total_C': 'beta', 'ieu-b-109': 'beta', 'ieu-b-110': 'beta', 'ieu-b-111': 'beta', 'ieu-b-25': 'beta', 'ieu-b-38': 'beta', 'ieu-b-39': 'beta', 'ukb-a-257': 'or', 'ukb-b-12064': 'or', 'ukb-b-17627': 'beta', 'ukb-b-18275': 'or', 'ukb-b-19953': 'beta', 'ukb-b-2209': 'beta', 'ukb-b-3957': 'beta', 'ukb-b-4424': 'beta', 'ukb-b-6134': 'beta',

In [11]:
# base_files
print(base_files)
for GWAS_ID in base_files:
    print(GWAS_ID)
    

{'ukb-b-13806': 'or', 'ukb-d-20405_0': 'or', 'ukb-d-20405_1': 'or', 'ukb-d-20405_2': 'or', 'met-d-Total_C': 'beta', 'ieu-b-109': 'beta', 'ieu-b-110': 'beta', 'ieu-b-111': 'beta', 'ieu-b-25': 'beta', 'ieu-b-38': 'beta', 'ieu-b-39': 'beta', 'ukb-a-257': 'or', 'ukb-b-12064': 'or', 'ukb-b-17627': 'beta', 'ukb-b-18275': 'or', 'ukb-b-19953': 'beta', 'ukb-b-2209': 'beta', 'ukb-b-3957': 'beta', 'ukb-b-4424': 'beta', 'ukb-b-6134': 'beta', 'ukb-b-6324': 'beta', 'ukb-b-7663': 'or', 'ukb-b-770': 'beta', 'ukb-b-8476': 'or', 'ukb-b-323': 'or', 'ukb-b-14699': 'or', 'ukb-b-14180': 'or', 'ukb-b-17243': 'or', 'ukb-b-6358': 'or', 'ukb-b-17006': 'or', 'ukb-b-5779': 'beta', 'ukb-b-15541': 'or', 'ukb-b-19732': 'or', 'ukb-b-20289': 'or', 'ukb-b-14057': 'or', 'ukb-b-12963': 'beta', 'ukb-b-12417': 'beta', 'ukb-b-11495': 'or', 'ukb-b-12493': 'or'}
ukb-b-13806
ukb-d-20405_0
ukb-d-20405_1
ukb-d-20405_2
met-d-Total_C
ieu-b-109
ieu-b-110
ieu-b-111
ieu-b-25
ieu-b-38
ieu-b-39
ukb-a-257
ukb-b-12064
ukb-b-17627
ukb-b-1

In [19]:
gender_map = {'Female': 0,'Male': 1}
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
def get_gender_and_age(PTID):
    gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
    age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
    return str(gender_map[gender]) + ' ' + str(age)

# NUM_TRAINING_SAMPLES = int(830 * .7)
# print('NUM_TRAINING_SAMPLES:', NUM_TRAINING_SAMPLES)
# f_writable = open('./COVAR_FILE.txt', 'w')
# f_TRAINING_SAMPLES = open('./TRAINING_SAMPLES.txt', 'w')
# with open('/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/covar_mds.txt') as f:
#     for i, line in enumerate(f):
#         if 'FID IID' in line:
#             line = line[:-1] + ' GENDER AGE\n'
#         else:
#             line = line[:-1] + ' ' + get_gender_and_age(PTID=line.split(' ')[1]) + '\n'
#         if i < NUM_TRAINING_SAMPLES:
#             f_TRAINING_SAMPLES.write(' '.join(line.split(' ')[:2])+'\n')
#         f_writable.write(line)
# #         print(line)
# f_writable.close()

FileNotFoundError: [Errno 2] No such file or directory: 'ADNIMERGE.csv'

In [8]:
def run_prsice(GWAS_ID, redo=False):
#     TARGET_DATA = '/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI' # old and smaller dataset
    #jei data diye ultimate train kora hocche model e
    #larger dataset er protita data er jnno protita trait er prscore ber kora hocche
    TARGET_DATA = '../larger_dataset/larger_dataset' # new and larger dataset
    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
#     need to alter between these two if needed
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/'):
#     if True:
        return_status = os.system(f'mkdir ./PRSice_output/{GWAS_ID}/')
        print('0:', return_status)
        if return_status == 256 and not redo:
            return return_status
    print('stat',{base_files[GWAS_ID]})
#     prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#         --dir ./PRSice_output \
#         --prsice ./PRSice_linux/PRSice_linux \
#         --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#         --target {TARGET_DATA} \
#         --thread 2 \
#         --stat {base_files[GWAS_ID].upper()} \
#         --{base_files[GWAS_ID]} \
#         --binary-target F \
#         --quantile 10 \
#         --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#         --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#         --score std \
#         --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#         --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'):
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
            --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    else :
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt '
#         print(prsice_command)
#         print('stat',{base_files[GWAS_ID]})
#         return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#         print('1:', return_status)
#         return_status = os.system('echo "===== Done ====="')
#         print('2:', return_status) 

    #             running second time with .valid extraction
#         prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#             --dir ./PRSice_output \
#             --prsice ./PRSice_linux/PRSice_linux \
#             --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#             --target {TARGET_DATA} \
#             --thread 2 \
#             --stat {base_files[GWAS_ID].upper()} \
#             --{base_files[GWAS_ID]} \
#             --binary-target F \
#             --quantile 10 \
#             --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#             --score std \
#             --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#             --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
        

# #     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
# #     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.validukb-b-12963
# #     print('1a:', return_status)
# #     return
# #     if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
# #         prsice_command += f' --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid' 
# #         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         print('1b:', return_status)
# #         if return_status != 0:
# # #             return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #             print('1c:', return_status)
# #     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
# #     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
# #     print('2:', return_status)
#     # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    print(prsice_command)
    print('stat',{base_files[GWAS_ID]})
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1:', return_status)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')
# https://github.com/adiamb/Remove-duplicate-snps-plink

# For ADNI Dataset

In [6]:
def run_prsice_for_adni(GWAS_ID, redo=False):
#     TARGET_DATA = '/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI' # old and smaller dataset

#     TARGET_DATA = '../larger_dataset/larger_dataset' # new and larger dataset

#     TARGET_DATA = "../../../GWA_tutorial-master/1_QC_GWAS/HapMap_3_r3_12" 
    TARGET_DATA = "../../../GWA_tutorial-master/2_Population_stratification/HapMap_3_r3_13" 
    
#     this dataset may need to be changed

    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
#     need to alter between these two if needed
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/'):
#     if True:
        return_status = os.system(f'mkdir ./PRSice_output/{GWAS_ID}/')
        print('0:', return_status)
        if return_status == 256 and not redo:
            return return_status
    print('stat',{base_files[GWAS_ID]})
#     prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#         --dir ./PRSice_output \
#         --prsice ./PRSice_linux/PRSice_linux \
#         --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#         --target {TARGET_DATA} \
#         --thread 2 \
#         --stat {base_files[GWAS_ID].upper()} \
#         --{base_files[GWAS_ID]} \
#         --binary-target F \
#         --quantile 10 \
#         --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#         --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#         --score std \
#         --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#         --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'):
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov "../../../GWA_tutorial-master/2_Population_stratification/covar_mds.txt" \
            --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    else :
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov "../../../GWA_tutorial-master/2_Population_stratification/covar_mds.txt" '
#         print(prsice_command)
#         print('stat',{base_files[GWAS_ID]})
#         return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#         print('1:', return_status)
#         return_status = os.system('echo "===== Done ====="')
#         print('2:', return_status) 

    #             running second time with .valid extraction
#         prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#             --dir ./PRSice_output \
#             --prsice ./PRSice_linux/PRSice_linux \
#             --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#             --target {TARGET_DATA} \
#             --thread 2 \
#             --stat {base_files[GWAS_ID].upper()} \
#             --{base_files[GWAS_ID]} \
#             --binary-target F \
#             --quantile 10 \
#             --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#             --score std \
#             --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#             --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
        

# #     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
# #     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.validukb-b-12963
# #     print('1a:', return_status)
# #     return
# #     if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
# #         prsice_command += f' --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid' 
# #         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         print('1b:', return_status)
# #         if return_status != 0:
# # #             return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #             print('1c:', return_status)
# #     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
# #     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
# #     print('2:', return_status)
#     # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    print(prsice_command)
    print('stat',{base_files[GWAS_ID]})
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1:', return_status)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')
# https://github.com/adiamb/Remove-duplicate-snps-plink

# For larger dataset generated by us

 Change the TARGET_DATA folder and also in the script change the cov source

In [40]:
def run_prsice_for_larger_by_us(GWAS_ID, redo=False):
#     TARGET_DATA = '/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI' # old and smaller dataset
    #jei data diye ultimate train kora hocche model e
    #larger dataset er protita data er jnno protita trait er prscore ber kora hocche
    TARGET_DATA = '../larger_dataset_by_naeem2/larger_dataset' # new and larger dataset
    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
#     need to alter between these two if needed
#     if not os.path.isfile(f'./PRSice_output/{GWAS_ID}/'):
#     if True:
#         return_status = os.system(f'mkdir ./PRSice_output/{GWAS_ID}/')
#         print('0:', return_status)
#         if return_status == 256 and not redo:
#             return return_status
    print('stat',{base_files[GWAS_ID]})
#     prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#         --dir ./PRSice_output \
#         --prsice ./PRSice_linux/PRSice_linux \
#         --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#         --target {TARGET_DATA} \
#         --thread 2 \
#         --stat {base_files[GWAS_ID].upper()} \
#         --{base_files[GWAS_ID]} \
#         --binary-target F \
#         --quantile 10 \
#         --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#         --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#         --score std \
#         --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#         --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'):
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 8 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt \
            --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    else :
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 8 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt '
#         print(prsice_command)
#         print('stat',{base_files[GWAS_ID]})
#         return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#         print('1:', return_status)
#         return_status = os.system('echo "===== Done ====="')
#         print('2:', return_status) 

    #             running second time with .valid extraction
#         prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#             --dir ./PRSice_output \
#             --prsice ./PRSice_linux/PRSice_linux \
#             --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#             --target {TARGET_DATA} \
#             --thread 2 \
#             --stat {base_files[GWAS_ID].upper()} \
#             --{base_files[GWAS_ID]} \
#             --binary-target F \
#             --quantile 10 \
#             --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#             --score std \
#             --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#             --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
        

# #     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
# #     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.validukb-b-12963
# #     print('1a:', return_status)
# #     return
# #     if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
# #         prsice_command += f' --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid' 
# #         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         print('1b:', return_status)
# #         if return_status != 0:
# # #             return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #             print('1c:', return_status)
# #     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
# #     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
# #     print('2:', return_status)
#     # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    print(prsice_command)
    print('stat',{base_files[GWAS_ID]})
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1:', return_status)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')
# https://github.com/adiamb/Remove-duplicate-snps-plink

# Notes on run_prsice

for each of the data ( individual ) that we have in target dataset, we use our built psrice framework to find out the prs value. so we now know the PRS for target dataset IDs.

In [42]:
# here for some files a weird error occurs: "reporter not initialized"
# soln : create corresponding directory
# bbj-a-78
# ukb-a-257
# bbj-a-46
# these are not included due to not being in the EUR ancestry
counter = 0
for GWAS_ID in base_files:
#     if GWAS_ID == 'ukb-b-6134':
    counter += 1
    if counter > 1:
        print('-----------------------counter ',counter,' ------------------------------------')
        print('*****GWAS ID IS*****',GWAS_ID)
    #         run_prsice(GWAS_ID=GWAS_ID)
        run_prsice_for_larger_by_us(GWAS_ID=GWAS_ID)

#         problem -> they do not have corresponding entries in prsice_output folder->create those folders manually

-----------------------counter  2  ------------------------------------
*****GWAS ID IS***** ukb-d-20405_0
stat {'or'}
Rscript ./PRSice_linux/PRSice.R             --dir ./PRSice_output             --prsice ./PRSice_linux/PRSice_linux             --base ./tabular_format_gwas_data/ukb-d-20405_0.tsv             --target ../larger_dataset_by_naeem2/larger_dataset             --thread 8             --stat OR             --or             --binary-target F             --quantile 10             --out ./PRSice_output/ukb-d-20405_0/ukb-d-20405_0             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-D-20405_0:ES --pvalue PVAL_generated_from_LP             --score std             --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt             --extract ./PRSice_output/ukb-d-20405_0/ukb-d-20405_0.valid
stat {'or'}


PRSice 2.3.3 (2020-08-05) 
https://github.com/choishingwan/PRSice
(C) 2016-2020 Shing Wan (Sam) Choi and Paul F. O'Reilly
GNU General Public License v3
If you use PRSice in any published work, please cite:
Choi SW, O'Reilly PF.
PRSice-2: Polygenic Risk Score Software for Biobank-Scale Data.
GigaScience 8, no. 7 (July 1, 2019)
2022-11-19 05:07:22
./PRSice_linux/PRSice_linux \
    --a1 ALT \
    --a2 REF \
    --bar-levels 0.001,0.05,0.1,0.2,0.3,0.4,0.5,1 \
    --base ./tabular_format_gwas_data/ukb-d-20405_0.tsv \
    --binary-target F \
    --bp POS \
    --chr CHROM \
    --clump-kb 250kb \
    --clump-p 1.000000 \
    --clump-r2 0.100000 \
    --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt \
    --extract ./PRSice_output/ukb-d-20405_0/ukb-d-20405_0.valid \
    --interval 5e-05 \
    --lower 5e-08 \
    --num-auto 22 \
    --or  \
    --out ./PRSice_output/ukb-d-20405_0/ukb-d-20405_0 \
    --pvalue PVAL_generated_from_LP \
    --score std \
    --seed 1294743028 \
    

Begin plotting
Current Rscript version = 2.3.3
Plotting the quantile plot
Plotting Bar Plot
Plotting the high resolution plot
1: 0
===== Done =====
2: 0
-----------------------counter  3  ------------------------------------
*****GWAS ID IS***** ukb-d-20405_1
stat {'or'}
Rscript ./PRSice_linux/PRSice.R             --dir ./PRSice_output             --prsice ./PRSice_linux/PRSice_linux             --base ./tabular_format_gwas_data/ukb-d-20405_1.tsv             --target ../larger_dataset_by_naeem2/larger_dataset             --thread 8             --stat OR             --or             --binary-target F             --quantile 10             --out ./PRSice_output/ukb-d-20405_1/ukb-d-20405_1             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-D-20405_1:ES --pvalue PVAL_generated_from_LP             --score std             --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt             --extract ./PRSice_output/ukb-d-20405_1/ukb-d-20405_1.valid
stat {'or'}


PRSice 2.3.3 (2020-08-05) 
https://github.com/choishingwan/PRSice
(C) 2016-2020 Shing Wan (Sam) Choi and Paul F. O'Reilly
GNU General Public License v3
If you use PRSice in any published work, please cite:
Choi SW, O'Reilly PF.
PRSice-2: Polygenic Risk Score Software for Biobank-Scale Data.
GigaScience 8, no. 7 (July 1, 2019)
2022-11-19 05:08:16
./PRSice_linux/PRSice_linux \
    --a1 ALT \
    --a2 REF \
    --bar-levels 0.001,0.05,0.1,0.2,0.3,0.4,0.5,1 \
    --base ./tabular_format_gwas_data/ukb-d-20405_1.tsv \
    --binary-target F \
    --bp POS \
    --chr CHROM \
    --clump-kb 250kb \
    --clump-p 1.000000 \
    --clump-r2 0.100000 \
    --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt \
    --extract ./PRSice_output/ukb-d-20405_1/ukb-d-20405_1.valid \
    --interval 5e-05 \
    --lower 5e-08 \
    --num-auto 22 \
    --or  \
    --out ./PRSice_output/ukb-d-20405_1/ukb-d-20405_1 \
    --pvalue PVAL_generated_from_LP \
    --score std \
    --seed 2008716090 \
    

Begin plotting
Current Rscript version = 2.3.3
Plotting the quantile plot
Plotting Bar Plot
Plotting the high resolution plot
1: 0
===== Done =====
2: 0
-----------------------counter  4  ------------------------------------
*****GWAS ID IS***** ukb-d-20405_2
stat {'or'}
Rscript ./PRSice_linux/PRSice.R             --dir ./PRSice_output             --prsice ./PRSice_linux/PRSice_linux             --base ./tabular_format_gwas_data/ukb-d-20405_2.tsv             --target ../larger_dataset_by_naeem2/larger_dataset             --thread 8             --stat OR             --or             --binary-target F             --quantile 10             --out ./PRSice_output/ukb-d-20405_2/ukb-d-20405_2             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-D-20405_2:ES --pvalue PVAL_generated_from_LP             --score std             --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt             --extract ./PRSice_output/ukb-d-20405_2/ukb-d-20405_2.valid
stat {'or'}


...

Begin plotting
Current Rscript version = 2.3.3
Plotting the quantile plot
Plotting Bar Plot
Plotting the high resolution plot
1: 0
===== Done =====
2: 0
-----------------------counter  38  ------------------------------------
*****GWAS ID IS***** ukb-b-11495
stat {'or'}
Rscript ./PRSice_linux/PRSice.R             --dir ./PRSice_output             --prsice ./PRSice_linux/PRSice_linux             --base ./tabular_format_gwas_data/ukb-b-11495.tsv             --target ../larger_dataset_by_naeem2/larger_dataset             --thread 8             --stat OR             --or             --binary-target F             --quantile 10             --out ./PRSice_output/ukb-b-11495/ukb-b-11495             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-B-11495:ES --pvalue PVAL_generated_from_LP             --score std             --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt             --extract ./PRSice_output/ukb-b-11495/ukb-b-11495.valid
stat {'or'}


PRSice 2.3.3 (2020-08-05) 
https://github.com/choishingwan/PRSice
(C) 2016-2020 Shing Wan (Sam) Choi and Paul F. O'Reilly
GNU General Public License v3
If you use PRSice in any published work, please cite:
Choi SW, O'Reilly PF.
PRSice-2: Polygenic Risk Score Software for Biobank-Scale Data.
GigaScience 8, no. 7 (July 1, 2019)
2022-11-19 05:43:23
./PRSice_linux/PRSice_linux \
    --a1 ALT \
    --a2 REF \
    --bar-levels 0.001,0.05,0.1,0.2,0.3,0.4,0.5,1 \
    --base ./tabular_format_gwas_data/ukb-b-11495.tsv \
    --binary-target F \
    --bp POS \
    --chr CHROM \
    --clump-kb 250kb \
    --clump-p 1.000000 \
    --clump-r2 0.100000 \
    --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt \
    --extract ./PRSice_output/ukb-b-11495/ukb-b-11495.valid \
    --interval 5e-05 \
    --lower 5e-08 \
    --num-auto 22 \
    --or  \
    --out ./PRSice_output/ukb-b-11495/ukb-b-11495 \
    --pvalue PVAL_generated_from_LP \
    --score std \
    --seed 4252418098 \
    --snp ID \

Begin plotting
Current Rscript version = 2.3.3
Plotting the quantile plot
Plotting Bar Plot
Plotting the high resolution plot
1: 0
===== Done =====
2: 0
-----------------------counter  39  ------------------------------------
*****GWAS ID IS***** ukb-b-12493
stat {'or'}
Rscript ./PRSice_linux/PRSice.R             --dir ./PRSice_output             --prsice ./PRSice_linux/PRSice_linux             --base ./tabular_format_gwas_data/ukb-b-12493.tsv             --target ../larger_dataset_by_naeem2/larger_dataset             --thread 8             --stat OR             --or             --binary-target F             --quantile 10             --out ./PRSice_output/ukb-b-12493/ukb-b-12493             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-B-12493:ES --pvalue PVAL_generated_from_LP             --score std             --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt             --extract ./PRSice_output/ukb-b-12493/ukb-b-12493.valid
stat {'or'}


PRSice 2.3.3 (2020-08-05) 
https://github.com/choishingwan/PRSice
(C) 2016-2020 Shing Wan (Sam) Choi and Paul F. O'Reilly
GNU General Public License v3
If you use PRSice in any published work, please cite:
Choi SW, O'Reilly PF.
PRSice-2: Polygenic Risk Score Software for Biobank-Scale Data.
GigaScience 8, no. 7 (July 1, 2019)
2022-11-19 05:44:10
./PRSice_linux/PRSice_linux \
    --a1 ALT \
    --a2 REF \
    --bar-levels 0.001,0.05,0.1,0.2,0.3,0.4,0.5,1 \
    --base ./tabular_format_gwas_data/ukb-b-12493.tsv \
    --binary-target F \
    --bp POS \
    --chr CHROM \
    --clump-kb 250kb \
    --clump-p 1.000000 \
    --clump-r2 0.100000 \
    --cov ../larger_dataset_by_naeem2/COVAR_FILE_bigger_dataset.txt \
    --extract ./PRSice_output/ukb-b-12493/ukb-b-12493.valid \
    --interval 5e-05 \
    --lower 5e-08 \
    --num-auto 22 \
    --or  \
    --out ./PRSice_output/ukb-b-12493/ukb-b-12493 \
    --pvalue PVAL_generated_from_LP \
    --score std \
    --seed 1195936060 \
    --snp ID \

Begin plotting
Current Rscript version = 2.3.3
Plotting the quantile plot
Plotting Bar Plot
Plotting the high resolution plot
1: 0
===== Done =====
2: 0


In [ ]:
# run_prsice(GWAS_ID='ukb-b-17627')

In [ ]:
!ls 'tabular_format_gwas_data/{GWAS_ID}.tsv'

In [3]:
# # need to remove this
# from glob import glob
# import pandas as pd
# category_map = {
#     'Continuous' : 'beta', 
#     'Categorical Ordered (assumed continuous)': 'beta',
#     'Binary': 'or',
#     'NA (Possibly binary)': 'or'
# }

# df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
# # all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
# # ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
# # ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257','ukb-b-2209','ukb-b-17627','ieub-109','ieu-b-110','ieu-b-111','met-d-Total_C'})
# # bbj-a-78,ukb-a-257,bbj-a-46
# # ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78'})
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78','ukb-b-13806','ukb-b-12064','ukb-b-323','ukb-b-14699','ukb-b-14699','ukb-b-5779'})
# print(len(ALL_GWAS_IDS))

# ALL_GWAS_IDS= {'ukb-b-17006','ukb-b-5779','ukb-b-15541','ukb-b-20289','ukb-b-19732','ukb-b-6324','ukb-b-7663','ukb-b-770','ukb-b-323','ukb-b-14699','ukb-b-14180','ukb-b-17243','ukb-b-6358','','ukb-b-8476','','ukb-b-6134','ukb-b-4424','ukb-b-3957','ukb-b-2209','ukb-b-19953','ukb-b-18275','ukb-b-17627','ukb-b-12064','ukb-a-257','ieu-b-39','ieu-b-38','ieu-b-25','ieu-b-111','ieu-b-110','ieu-b-109','met-d-Total_C','ukb-d-20405_2','ukb-d-20405_1','ukb-d-20405_0','ukb-b-13806','ukb-b-13806','ukb-b-12064','ukb-b-323','ukb-b-14699','ukb-b-14699','ukb-b-5779'}

# base_files = {}

# # base_files=[] #naeem mod
# for GWAS_ID in ALL_GWAS_IDS:

# #     file = ROOT + GWAS_ID + '.vcf.gz'
#     base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]]

# #     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)
# # print(name_mapping)
# # print(name_mapping)

# gender_map = {'Female': 0,'Male': 1}
# ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
# def get_gender_and_age(PTID):
#     gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
#     age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
#     return str(gender_map[gender]) + ' ' + str(age)


29


IndexError: index 0 is out of bounds for axis 0 with size 0

In [43]:
base_files
GWAS_ID = 'ukb-b-14057'

#====

def get_prs_values(GWAS_ID):
    prsice_output = f'./PRSice_output/{GWAS_ID}/'

    if False:
        with open(prsice_output+f'{GWAS_ID}.all_score', 'r') as f:
            lines = f.readlines()
            lines = ['\t'.join(x.split()) for x in lines]
            lines = '\n'.join(lines)

        with open(prsice_output+f'{GWAS_ID}.all_score.tsv', 'w') as f:
            f.write(lines)

        # best_p_val_threshold = '0.00025005'
        best_p_val_threshold = str(open(prsice_output+f'{GWAS_ID}.summary').readlines()[1].split('\t')[2]) 
    #     print(best_p_val_threshold) 
    prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy() 
    return prs_array

In [5]:
get_prs_values(GWAS_ID=GWAS_ID).shape[0]

/tmp/ipykernel_16760/1595037025.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  get_prs_values(GWAS_ID=GWAS_ID).shape[0]


1816

In [44]:

import os
folderCounter=14
folderCounter=folderCounter+1
# inputFolder="input/"+str(folderCounter)+"/"
inputFolder=""
# os.mkdir(inputFolder)


In [45]:
import numpy as np
import os.path
import json
name_mapping={}
# GWAS_ID
# ukb-b-13806
# ukb-b-12064
# ukb-b-323
# ukb-b-14699
# ukb-b-5779
# base_files=base_files-
print('base files',base_files)
newBaseFiles=[]
for GWAS_ID in base_files:
    prsice_output = f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best'
    if os.path.exists(prsice_output):  
        newBaseFiles.append(GWAS_ID)
print('new base files',newBaseFiles)

PRS_feature_matrix = np.zeros([len(newBaseFiles), 1816])
# PRS_feature_matrix = np.zeros([len(newBaseFiles), 846])

for i, GWAS_ID in enumerate(newBaseFiles):
    print(GWAS_ID)
    prsice_output = f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best'
    if os.path.exists(prsice_output):  
        PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
        name_mapping[GWAS_ID] =df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 1]
    else :
        print("no path exists")
        
# with open(inputFolder+"traits_map.json", "w") as outfile:
with open("traits_map.json", "w") as outfile:
    
    json.dump(name_mapping, outfile)

    

base files {'ukb-b-13806': 'or', 'ukb-d-20405_0': 'or', 'ukb-d-20405_1': 'or', 'ukb-d-20405_2': 'or', 'met-d-Total_C': 'beta', 'ieu-b-109': 'beta', 'ieu-b-110': 'beta', 'ieu-b-111': 'beta', 'ieu-b-25': 'beta', 'ieu-b-38': 'beta', 'ieu-b-39': 'beta', 'ukb-a-257': 'or', 'ukb-b-12064': 'or', 'ukb-b-17627': 'beta', 'ukb-b-18275': 'or', 'ukb-b-19953': 'beta', 'ukb-b-2209': 'beta', 'ukb-b-3957': 'beta', 'ukb-b-4424': 'beta', 'ukb-b-6134': 'beta', 'ukb-b-6324': 'beta', 'ukb-b-7663': 'or', 'ukb-b-770': 'beta', 'ukb-b-8476': 'or', 'ukb-b-323': 'or', 'ukb-b-14699': 'or', 'ukb-b-14180': 'or', 'ukb-b-17243': 'or', 'ukb-b-6358': 'or', 'ukb-b-17006': 'or', 'ukb-b-5779': 'beta', 'ukb-b-15541': 'or', 'ukb-b-19732': 'or', 'ukb-b-20289': 'or', 'ukb-b-14057': 'or', 'ukb-b-12963': 'beta', 'ukb-b-12417': 'beta', 'ukb-b-11495': 'or', 'ukb-b-12493': 'or'}
new base files ['ukb-b-13806', 'ukb-d-20405_0', 'ukb-d-20405_1', 'ukb-d-20405_2', 'met-d-Total_C', 'ieu-b-109', 'ieu-b-110', 'ieu-b-111', 'ieu-b-25', 'ieu-

/tmp/ipykernel_12328/1739813349.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy()
/tmp/ipykernel_12328/1739813349.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy()
/tmp/ipykernel_12328/1739813349.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy()
/tmp/ipykernel_12328/1739813349.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  prs_array = pd.read_csv(prsice_output+f'{GWA

In [46]:
PRS_feature_matrix = PRS_feature_matrix.T
np.save(inputFolder+'PRS_feature_matrix', PRS_feature_matrix)

In [47]:
PRS_feature_matrix.shape
import random

In [48]:
TRAIN_SAMPLE_SIZE = 550
train_samples = list(range(1816))
random.shuffle(train_samples)
train_samples, test_samples = train_samples[:TRAIN_SAMPLE_SIZE], train_samples[TRAIN_SAMPLE_SIZE:]
print(test_samples)
print(len(train_samples), len(test_samples), len(train_samples) + len(test_samples))
# PRS_feature_matrix[train_samples].mean(0), PRS_feature_matrix[train_samples].std(0)

[914, 1589, 500, 1620, 1260, 1759, 482, 1317, 1700, 436, 1547, 1697, 888, 1045, 271, 144, 920, 385, 611, 249, 340, 1012, 1143, 1164, 919, 1651, 1635, 1537, 406, 878, 605, 886, 968, 861, 1500, 1484, 416, 1172, 534, 1053, 1202, 670, 516, 1015, 1116, 851, 891, 496, 1679, 1430, 1030, 1609, 1174, 995, 844, 127, 368, 1556, 1236, 88, 590, 132, 1420, 1208, 1475, 1517, 1684, 758, 880, 469, 12, 1585, 660, 211, 1627, 1719, 657, 1743, 1111, 606, 1810, 780, 170, 1314, 1487, 1076, 242, 831, 1442, 27, 1005, 180, 1705, 717, 1306, 739, 1622, 60, 671, 30, 169, 1136, 841, 1061, 638, 1319, 163, 1805, 1001, 1166, 1300, 837, 1189, 1263, 535, 405, 1675, 494, 597, 802, 707, 234, 458, 44, 897, 1206, 1227, 1542, 579, 403, 31, 1707, 410, 607, 585, 367, 1125, 501, 1521, 59, 447, 1417, 75, 817, 1513, 1105, 1100, 539, 917, 1595, 687, 859, 877, 1477, 103, 83, 1289, 787, 1040, 1405, 1795, 722, 703, 41, 1390, 1775, 1617, 139, 280, 1503, 1815, 1472, 1063, 1064, 505, 1029, 1545, 613, 836, 875, 1386, 1415, 344, 1766, 286

In [49]:
PRS_feature_matrix[test_samples].mean(0), PRS_feature_matrix[test_samples].std(0)

(array([-0.01031536, -0.02281834, -0.00394759, -0.02264448, -0.0193774 ,
         0.00632123, -0.00650544, -0.00742301, -0.02755314,  0.01118149,
        -0.00981811, -0.02792764, -0.00815029,  0.00993923, -0.0237625 ,
        -0.00446245,  0.00538661,  0.01710965,  0.01374963,  0.010898  ,
        -0.00243885, -0.02328532, -0.01074045, -0.01649424,  0.00676362,
         0.00941212, -0.02498994, -0.01983584, -0.02850219, -0.03180173,
        -0.04444718, -0.01293681, -0.0255557 , -0.01489494, -0.02831349,
        -0.0313447 , -0.02836364, -0.0244011 , -0.02474088]),
 array([0.96742415, 0.91450364, 0.97581383, 0.92597209, 1.01119581,
        0.99183426, 0.99871367, 1.00089391, 1.00691625, 0.9990749 ,
        1.00174667, 0.92199834, 0.97453648, 0.9970049 , 0.93051232,
        1.01008579, 1.01134924, 1.01171729, 0.93070884, 0.99717131,
        1.00201104, 0.9215018 , 1.00674744, 0.96450593, 0.99960003,
        1.01833358, 0.95689942, 0.9501297 , 0.96951256, 0.98937893,
        0.90405369,

In [50]:
PRS_feature_matrix[:].mean(0), PRS_feature_matrix[:].std(0)

(array([ 1.80121253e-11, -7.23843658e-11, -4.03398744e-11, -1.24741255e-11,
         1.19856782e-11,  8.02313086e-12, -3.06668484e-11,  6.21563858e-11,
        -2.72191628e-11,  7.99063703e-12,  7.19383295e-11, -1.15251910e-12,
        -3.74664176e-11,  1.85110097e-11, -5.24669764e-11,  1.11084739e-11,
         5.59416361e-11,  3.21943862e-11, -5.24559427e-11, -2.76734093e-12,
        -1.95044098e-11,  9.72282264e-12,  5.42070353e-12,  6.10582832e-11,
         3.02257739e-11,  1.54763251e-11,  3.52919065e-11, -6.37114248e-12,
        -8.16188233e-12,  4.53524247e-11, -1.82294982e-11,  3.02863363e-12,
         6.98566804e-12,  3.05832070e-11, -1.58479201e-13,  4.16432195e-11,
         2.62714803e-11,  1.76151057e-11, -4.53576677e-12]),
 array([0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972

In [51]:

import json
import pandas as pd
GWAS_ID = 'ukb-b-13806'
usable_samples_ADNI_unsplitted = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {'_'.join(sample.split('_')[1:]):sample for sample in (usable_samples_ADNI_unsplitted)}
# above line may need to be commented out
# json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))
# print(type(usable_samples_ADNI), len(usable_samples_ADNI))
usable_samples_ADNI

/tmp/ipykernel_12328/1558320868.py:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  usable_samples_ADNI_unsplitted = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()


{'036_S_6231': 'ADNI3_036_S_6231',
 '006_S_6277': 'ADNI3_006_S_6277',
 '129_S_6146': 'ADNI3_129_S_6146',
 '033_S_6352': 'ADNI3_033_S_6352',
 '027_S_6183': 'ADNI3_027_S_6183',
 '005_S_6427': 'ADNI3_005_S_6427',
 '127_S_6147': 'ADNI3_127_S_6147',
 '114_S_6251': 'ADNI3_114_S_6251',
 '129_S_6228': 'ADNI3_129_S_6228',
 '114_S_6309': 'ADNI3_114_S_6309',
 '135_S_6110': 'ADNI3_135_S_6110',
 '020_S_6358': 'ADNI3_020_S_6358',
 '135_S_6411': 'ADNI3_135_S_6411',
 '024_S_6202': 'ADNI3_024_S_6202',
 '018_S_6414': 'ADNI3_018_S_6414',
 '002_S_6103': 'ADNI3_002_S_6103',
 '177_S_6408': 'ADNI3_177_S_6408',
 '014_S_6148': 'ADNI3_014_S_6148',
 '036_S_6466': 'ADNI3_036_S_6466',
 '036_S_6134': 'ADNI3_036_S_6134',
 '007_S_6455': 'ADNI3_007_S_6455',
 '037_S_6271': 'ADNI3_037_S_6271',
 '116_S_6100': 'ADNI3_116_S_6100',
 '027_S_6327': 'ADNI3_027_S_6327',
 '099_S_6097': 'ADNI3_099_S_6097',
 '127_S_6330': 'ADNI3_127_S_6330',
 '127_S_6168': 'ADNI3_127_S_6168',
 '018_S_6351': 'ADNI3_018_S_6351',
 '009_S_6212': 'ADNI

In [23]:
!ls PRSice_output

'25_7_22_thyroid traits.zip'   ukb-a-257     ukb-b-17006   ukb-b-5779
 bbj-a-46		       ukb-b-11495   ukb-b-17243   ukb-b-6134
 bbj-a-78		       ukb-b-12064   ukb-b-17627   ukb-b-6324
 ieu-b-109		       ukb-b-12417   ukb-b-18275   ukb-b-6358
 ieu-b-110		       ukb-b-12493   ukb-b-19732   ukb-b-7663
 ieu-b-111		       ukb-b-12963   ukb-b-19953   ukb-b-770
 ieu-b-25		       ukb-b-13806   ukb-b-20289   ukb-b-8476
 ieu-b-38		       ukb-b-14057   ukb-b-2209    ukb-d-20405_0
 ieu-b-39		       ukb-b-14180   ukb-b-323	   ukb-d-20405_1
 lib			       ukb-b-14699   ukb-b-3957    ukb-d-20405_2
 met-d-Total_C		       ukb-b-15541   ukb-b-4424


In [52]:
import json
import numpy as np
# PRS_feature_matrix = np.load('./PRS_feature_matrix.npy')
PRS_feature_matrix = np.load(inputFolder+'PRS_feature_matrix.npy')
# usable_samples_ADNI = json.load(open('./usable_samples_ADNI.json'))
print(PRS_feature_matrix.shape, usable_samples_ADNI.__len__())

# usable_samples_ADNI

(1816, 39) 1816


# prsice result used in our model
usable_samples_adni-> from .best of prsice corresponding to larger dataset we get the IIDs ( ie the individuals that we considered for our studies )
now , larger data set is actually a part of ADNIMERGE dataset . So, here we have all the considered patients, but some of the data that are in ADNIMERGE are not present in larger dataset. That leads to the check of "sample in usable_samples_adni". 
The reason behind this is related to the generation of "larger dataset" generation from "ADNI" dataset.
Here our main task is to find out those samples that have corresponding .best file entry, ie included in larger dataset.

In [55]:
# samples with no restriction
THRESHOLD_MONTH = 12*0
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
print(ADNIMERGE.head())
print(ADNIMERGE.shape)
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
print("Dementia shape : ", Dementia.shape)
print(Dementia)
Samples_Dementia = set(Dementia['PTID'].unique())
print("Samples Dementia length : ", len(Samples_Dementia) )
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
print("Samples NonDementia length : ", len( Samples_NonDementia ) )

# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    
    if True:
#     if last_month_of_dx >= THRESHOLD_MONTH :
#         if last_dx != 'Dementia':
#             print(last_dx)
        
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
            
            
            Final_Samples_Dementia = Final_Samples_Dementia.union({usable_samples_ADNI[sample]})
            
        
#             why check for dementia again?
                

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
#     if sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {usable_samples_ADNI[sample]} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]
print("Final Samples Dementia length : ", len(Final_Samples_Dementia) )
print("Final Samples Non-Dementia length : ", len(Final_Samples_NonDementia) )

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
# json.dump(Final_Samples, open(inputFolder+'Final_Samples.json', 'w'))
json.dump(Final_Samples, open('Final_Samples_nores.json', 'w'))
json.dump(Final_Samples_Dementia, open('Final_Samples_Dementia.json', 'w'))
json.dump(Final_Samples_NonDementia, open('Final_Samples_NonDementia.json', 'w'))


Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

/tmp/ipykernel_12328/2187409031.py:3: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


   RID        PTID VISCODE  SITE COLPROT ORIGPROT  EXAMDATE DX_bl   AGE  \
0    2  011_S_0002      bl    11   ADNI1    ADNI1  08-09-05    CN  74.3   
1    3  011_S_0003      bl    11   ADNI1    ADNI1  12-09-05    AD  81.3   
2    3  011_S_0003     m06    11   ADNI1    ADNI1  13-03-06    AD  81.3   
3    3  011_S_0003     m12    11   ADNI1    ADNI1  12-09-06    AD  81.3   
4    3  011_S_0003     m24    11   ADNI1    ADNI1  12-09-07    AD  81.3   

  PTGENDER  ...  TAU_bl PTAU_bl   FDG_bl PIB_bl  AV45_bl  Years_bl  Month_bl  \
0     Male  ...     NaN     NaN  1.36665    NaN      NaN  0.000000   0.00000   
1     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.000000   0.00000   
2     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.498289   5.96721   
3     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.999316  11.96720   
4     Male  ...   239.7   22.83  1.08355    NaN      NaN  1.998630  23.93440   

   Month     M update_stamp  
0      0   0.0      55:41.0  
1      0

(654, 1144, 1798)

In [56]:
# 2 years samples
THRESHOLD_MONTH = 12*2
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
print(ADNIMERGE.head())
print(ADNIMERGE.shape)
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
print("Dementia shape : ", Dementia.shape)
print(Dementia)
Samples_Dementia = set(Dementia['PTID'].unique())
print("Samples Dementia length : ", len(Samples_Dementia) )
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
print("Samples NonDementia length : ", len( Samples_NonDementia ) )

# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    
    if True:
#     if last_month_of_dx >= THRESHOLD_MONTH :
#         if last_dx != 'Dementia':
#             print(last_dx)
        
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
            
            
            Final_Samples_Dementia = Final_Samples_Dementia.union({usable_samples_ADNI[sample]})
            
        
#             why check for dementia again?
                

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
#     if sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {usable_samples_ADNI[sample]} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]
print("Final Samples Dementia length : ", len(Final_Samples_Dementia) )
print("Final Samples Non-Dementia length : ", len(Final_Samples_NonDementia) )

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
# json.dump(Final_Samples, open(inputFolder+'Final_Samples.json', 'w'))
json.dump(Final_Samples, open('Final_Samples_2yrs.json', 'w'))
json.dump(Final_Samples_Dementia, open('Final_Samples_Dementia.json', 'w'))
json.dump(Final_Samples_NonDementia, open('Final_Samples_NonDementia.json', 'w'))


Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

/tmp/ipykernel_12328/634977319.py:3: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


   RID        PTID VISCODE  SITE COLPROT ORIGPROT  EXAMDATE DX_bl   AGE  \
0    2  011_S_0002      bl    11   ADNI1    ADNI1  08-09-05    CN  74.3   
1    3  011_S_0003      bl    11   ADNI1    ADNI1  12-09-05    AD  81.3   
2    3  011_S_0003     m06    11   ADNI1    ADNI1  13-03-06    AD  81.3   
3    3  011_S_0003     m12    11   ADNI1    ADNI1  12-09-06    AD  81.3   
4    3  011_S_0003     m24    11   ADNI1    ADNI1  12-09-07    AD  81.3   

  PTGENDER  ...  TAU_bl PTAU_bl   FDG_bl PIB_bl  AV45_bl  Years_bl  Month_bl  \
0     Male  ...     NaN     NaN  1.36665    NaN      NaN  0.000000   0.00000   
1     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.000000   0.00000   
2     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.498289   5.96721   
3     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.999316  11.96720   
4     Male  ...   239.7   22.83  1.08355    NaN      NaN  1.998630  23.93440   

   Month     M update_stamp  
0      0   0.0      55:41.0  
1      0

(654, 867, 1521)

In [57]:
# 4 years samples
THRESHOLD_MONTH = 12*4
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
print(ADNIMERGE.head())
print(ADNIMERGE.shape)
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
print("Dementia shape : ", Dementia.shape)
print(Dementia)
Samples_Dementia = set(Dementia['PTID'].unique())
print("Samples Dementia length : ", len(Samples_Dementia) )
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
print("Samples NonDementia length : ", len( Samples_NonDementia ) )

# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    
    if True:
#     if last_month_of_dx >= THRESHOLD_MONTH :
#         if last_dx != 'Dementia':
#             print(last_dx)
        
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
            
            
            Final_Samples_Dementia = Final_Samples_Dementia.union({usable_samples_ADNI[sample]})
            
        
#             why check for dementia again?
                

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
#     if sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {usable_samples_ADNI[sample]} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]
print("Final Samples Dementia length : ", len(Final_Samples_Dementia) )
print("Final Samples Non-Dementia length : ", len(Final_Samples_NonDementia) )

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
# json.dump(Final_Samples, open(inputFolder+'Final_Samples.json', 'w'))
json.dump(Final_Samples, open('Final_Samples_4yrs.json', 'w'))
json.dump(Final_Samples_Dementia, open('Final_Samples_Dementia.json', 'w'))
json.dump(Final_Samples_NonDementia, open('Final_Samples_NonDementia.json', 'w'))


Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

/tmp/ipykernel_12328/2450854249.py:3: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


   RID        PTID VISCODE  SITE COLPROT ORIGPROT  EXAMDATE DX_bl   AGE  \
0    2  011_S_0002      bl    11   ADNI1    ADNI1  08-09-05    CN  74.3   
1    3  011_S_0003      bl    11   ADNI1    ADNI1  12-09-05    AD  81.3   
2    3  011_S_0003     m06    11   ADNI1    ADNI1  13-03-06    AD  81.3   
3    3  011_S_0003     m12    11   ADNI1    ADNI1  12-09-06    AD  81.3   
4    3  011_S_0003     m24    11   ADNI1    ADNI1  12-09-07    AD  81.3   

  PTGENDER  ...  TAU_bl PTAU_bl   FDG_bl PIB_bl  AV45_bl  Years_bl  Month_bl  \
0     Male  ...     NaN     NaN  1.36665    NaN      NaN  0.000000   0.00000   
1     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.000000   0.00000   
2     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.498289   5.96721   
3     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.999316  11.96720   
4     Male  ...   239.7   22.83  1.08355    NaN      NaN  1.998630  23.93440   

   Month     M update_stamp  
0      0   0.0      55:41.0  
1      0

(654, 504, 1158)

In [58]:
# This struggles in our ADNI dataset
THRESHOLD_MONTH = 12*6
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
print(ADNIMERGE.head())
print(ADNIMERGE.shape)
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
print("Dementia shape : ", Dementia.shape)
print(Dementia)
Samples_Dementia = set(Dementia['PTID'].unique())
print("Samples Dementia length : ", len(Samples_Dementia) )
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
print("Samples NonDementia length : ", len( Samples_NonDementia ) )

# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    
    if True:
#     if last_month_of_dx >= THRESHOLD_MONTH :
#         if last_dx != 'Dementia':
#             print(last_dx)
        
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
            
            
            Final_Samples_Dementia = Final_Samples_Dementia.union({usable_samples_ADNI[sample]})
            
        
#             why check for dementia again?
                

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
#     if sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {usable_samples_ADNI[sample]} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]
print("Final Samples Dementia length : ", len(Final_Samples_Dementia) )
print("Final Samples Non-Dementia length : ", len(Final_Samples_NonDementia) )

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
# json.dump(Final_Samples, open(inputFolder+'Final_Samples.json', 'w'))
json.dump(Final_Samples, open('Final_Samples_6yrs.json', 'w'))
json.dump(Final_Samples_Dementia, open('Final_Samples_Dementia.json', 'w'))
json.dump(Final_Samples_NonDementia, open('Final_Samples_NonDementia.json', 'w'))


Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

/tmp/ipykernel_12328/1020319883.py:3: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


   RID        PTID VISCODE  SITE COLPROT ORIGPROT  EXAMDATE DX_bl   AGE  \
0    2  011_S_0002      bl    11   ADNI1    ADNI1  08-09-05    CN  74.3   
1    3  011_S_0003      bl    11   ADNI1    ADNI1  12-09-05    AD  81.3   
2    3  011_S_0003     m06    11   ADNI1    ADNI1  13-03-06    AD  81.3   
3    3  011_S_0003     m12    11   ADNI1    ADNI1  12-09-06    AD  81.3   
4    3  011_S_0003     m24    11   ADNI1    ADNI1  12-09-07    AD  81.3   

  PTGENDER  ...  TAU_bl PTAU_bl   FDG_bl PIB_bl  AV45_bl  Years_bl  Month_bl  \
0     Male  ...     NaN     NaN  1.36665    NaN      NaN  0.000000   0.00000   
1     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.000000   0.00000   
2     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.498289   5.96721   
3     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.999316  11.96720   
4     Male  ...   239.7   22.83  1.08355    NaN      NaN  1.998630  23.93440   

   Month     M update_stamp  
0      0   0.0      55:41.0  
1      0

(654, 320, 974)

In [59]:
import json
import pandas as pd
GWAS_ID = 'ukb-b-14057'
usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {sample:idx for idx,sample in enumerate(usable_samples_ADNI)}
json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))
# json.dump(usable_samples_ADNI, open(inputFolder+'usable_samples_ADNI.json', 'w'))

usable_samples_ADNI

/tmp/ipykernel_12328/3847962530.py:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()


{'ADNI3_036_S_6231': 0,
 'ADNI3_006_S_6277': 1,
 'ADNI3_129_S_6146': 2,
 'ADNI3_033_S_6352': 3,
 'ADNI3_027_S_6183': 4,
 'ADNI3_005_S_6427': 5,
 'ADNI3_127_S_6147': 6,
 'ADNI3_114_S_6251': 7,
 'ADNI3_129_S_6228': 8,
 'ADNI3_114_S_6309': 9,
 'ADNI3_135_S_6110': 10,
 'ADNI3_020_S_6358': 11,
 'ADNI3_135_S_6411': 12,
 'ADNI3_024_S_6202': 13,
 'ADNI3_018_S_6414': 14,
 'ADNI3_002_S_6103': 15,
 'ADNI3_177_S_6408': 16,
 'ADNI3_014_S_6148': 17,
 'ADNI3_036_S_6466': 18,
 'ADNI3_036_S_6134': 19,
 'ADNI3_007_S_6455': 20,
 'ADNI3_037_S_6271': 21,
 'ADNI3_116_S_6100': 22,
 'ADNI3_027_S_6327': 23,
 'ADNI3_099_S_6097': 24,
 'ADNI3_127_S_6330': 25,
 'ADNI3_127_S_6168': 26,
 'ADNI3_018_S_6351': 27,
 'ADNI3_009_S_6212': 28,
 'ADNI3_168_S_6180': 29,
 'ADNI3_116_S_6119': 30,
 'ADNI3_023_S_6346': 31,
 'ADNI3_168_S_6065': 32,
 'ADNI3_035_S_6200': 33,
 'ADNI3_023_S_6399': 34,
 'ADNI3_037_S_6125': 35,
 'ADNI3_116_S_6428': 36,
 'ADNI3_041_S_6192': 37,
 'ADNI3_941_S_6333': 38,
 'ADNI3_006_S_6209': 39,
 'ADNI3_03

In [61]:
ALL_GWAS_IDS = [key for key in base_files]
print(ALL_GWAS_IDS)

['ukb-b-13806', 'ukb-d-20405_0', 'ukb-d-20405_1', 'ukb-d-20405_2', 'met-d-Total_C', 'ieu-b-109', 'ieu-b-110', 'ieu-b-111', 'ieu-b-25', 'ieu-b-38', 'ieu-b-39', 'ukb-a-257', 'ukb-b-12064', 'ukb-b-17627', 'ukb-b-18275', 'ukb-b-19953', 'ukb-b-2209', 'ukb-b-3957', 'ukb-b-4424', 'ukb-b-6134', 'ukb-b-6324', 'ukb-b-7663', 'ukb-b-770', 'ukb-b-8476', 'ukb-b-323', 'ukb-b-14699', 'ukb-b-14180', 'ukb-b-17243', 'ukb-b-6358', 'ukb-b-17006', 'ukb-b-5779', 'ukb-b-15541', 'ukb-b-19732', 'ukb-b-20289', 'ukb-b-14057', 'ukb-b-12963', 'ukb-b-12417', 'ukb-b-11495', 'ukb-b-12493']


In [ ]:
df

In [62]:
df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
all_traits = []
for GWAS_ID in ALL_GWAS_IDS:
    trait = str(df[df['GWAS_ID'] == GWAS_ID]['Trait'].to_numpy()[0])
    print(GWAS_ID, ':', trait)
    all_traits += [trait]
print(all_traits)

ukb-b-13806 : Non-cancer illness code, self-reported: type 2 diabetes
ukb-d-20405_0 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: No
ukb-d-20405_1 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, but not in the last year
ukb-d-20405_2 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, during the last year
met-d-Total_C : Total cholesterol
ieu-b-109 : HDL cholesterol
ieu-b-110 : LDL cholesterol
ieu-b-111 : triglycerides
ieu-b-25 : Cigarettes per Day
ieu-b-38 : systolic blood pressure
ieu-b-39 : diastolic blood pressure
ukb-a-257 : Hearing difficulty/problems: Yes
ukb-b-12064 : Non-cancer illness code, self-reported: depression
ukb-b-17627 : Non-oily fish intake
ukb-b-18275 : Hearing difficulty/problems with background noise
ukb-b-19953 : Body mass index (BMI)
ukb-b-2209 : Oily fish intake
ukb-b-3957 : Sleeplessness / insomnia
ukb-b-4424 : Sleep duration
ukb-

In [ ]:
# import torch
# torch.__version__

In [ ]:
# # this is for demo purpose
# prsice_command = f'Rscript ./PRSice_linux/PRSice.R --dir ./PRSice_linux	\
# --prsice ./PRSice_linux/PRSice_linux     \
# --base ./PRSice_linux/simple_table_with_pval.txt     \
# --target ./PRSice_linux/ADNI_plink     \
# --thread 1     \
# --beta     \
# --binary-target F	\
# --out ./PRSice_output/PRSice	\
# --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-b-6134:ES --pvalue PVAL_generated_from_LP	\
# --extract ./PRSice_output/PRSice.valid'

In [ ]:
#     # this is for demo purpose
#     import os
#     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#     print('1a:', return_status)